### Consigna: Si la experiencia de un usuario es la cantidad de logs en los que participó, queremos saber que tanto nos sirve para predecir el futuro vandalismo: ¿Cuál es la probabilidad de que un usuario sea bloqueado según experiencias: <10, 10-40, 40-100, >100? Tener en cuenta que esta experiencia debe ser PREVIA al bloqueo del usuario (⭐⭐⭐)

Importo la libreria que utilizare en el desarrollo de esta consulta.

In [1]:
import pandas as pd

In [2]:
archivo = pd.read_csv('logs.csv',\
                     usecols= ['item_id','timestamp','logtype','contributor_ip','contributor_username','title'])

C:\Users\julip\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
archivo['Usuarios'] = archivo['contributor_username'].fillna(archivo['contributor_ip'])

In [4]:
#Me quedo con la parte del dataFrame que me interesa
arc=archivo[['Usuarios','item_id','timestamp','logtype','title']]

In [5]:
#libero memoria RAM
del archivo

In [6]:
arc.isnull().sum()

Usuarios     1077
item_id         0
timestamp       0
logtype         2
title        2005
dtype: int64

In [7]:
arc.dropna(inplace=True)

In [8]:
arc.isnull().sum()

Usuarios     0
item_id      0
timestamp    0
logtype      0
title        0
dtype: int64

In [9]:
totalUsuarios=arc.groupby(['Usuarios']).agg({'Usuarios':'count'})
totalUsuarios

,Usuarios
Usuarios,
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,1
! ! ! Histoire hebdomadaire de la patrie ! ! !,1
! Bikkit !,1
! DanSkammelsrod !,1
! Gabriel !,1
...,...
𝚕𝚒𝚝𝚢 𝚌𝚟18,1
😂,1
🤖,1


In [10]:
totalUsuarios.rename(columns={'Usuarios':'Experiencia'},inplace=True)

In [11]:
totalUsuarios.reset_index(inplace=True)

In [12]:
def asignarCat(x):
    if x < 10:
        return ("<10")
    elif 10<x<40:
        return("10-40")
    elif 40<x<100:
        return("40-100")
    else:
        return (">100")

In [13]:
totalUsuarios['categoria'] = totalUsuarios['Experiencia'].apply(asignarCat)
totalUsuarios

,Usuarios,Experiencia,categoria
0,! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,1,<10
1,! ! ! Histoire hebdomadaire de la patrie ! ! !,1,<10
2,! Bikkit !,1,<10
3,! DanSkammelsrod !,1,<10
4,! Gabriel !,1,<10
...,...,...,...
6272842,𝚕𝚒𝚝𝚢 𝚌𝚟18,1,<10
6272843,😂,1,<10
6272844,🤖,1,<10
6272845,𠔻𪚥,1,<10


In [14]:
totalUsuarios['categoria'].value_counts()

<10       6262394
10-40        5926
>100         2989
40-100       1538
Name: categoria, dtype: int64

Trabajo con los bloqueos

In [15]:
bloqueos = arc.groupby('logtype').get_group('block')
bloqueos

,Usuarios,item_id,timestamp,logtype,title
5,Dodo,6,2004-12-23T08:28:32Z,block,Usuario:212.55.30.157
287,Dodo,288,2004-12-27T17:54:50Z,block,Usuario:201.241.228.15
288,Dodo,289,2004-12-27T17:57:12Z,block,Usuario:201.255.50.32
290,Dodo,291,2004-12-27T17:59:56Z,block,Usuario:84.122.0.158
313,Comae,314,2004-12-28T01:38:59Z,block,Usuario:81.192.12.93
...,...,...,...,...,...
12188865,Taichi,15587319,2021-09-03T02:05:46Z,block,Usuario:186.141.200.121
12188886,Hasley,15587340,2021-09-03T02:24:41Z,block,Usuario:Galaxi066
12188909,Marcelo,15587363,2021-09-03T02:52:42Z,block,Usuario:Xx.tkpro.xx
12188924,Hasley,15587378,2021-09-03T03:00:14Z,block,Usuario:181.167.104.152


In [16]:
bloqueos = bloqueos.rename(columns={'timestamp':'fechaBloqueo'})
bloqueos = bloqueos.astype({'title': 'str'})

In [17]:
bloqueos['UsuarioBloqueado'] = bloqueos['title'].apply(lambda x: x.split(':')[1])

In [18]:
bloqueos = bloqueos[['UsuarioBloqueado','fechaBloqueo']]

In [19]:
df_merge = arc.merge(bloqueos,left_on='Usuarios',right_on='UsuarioBloqueado',how='inner')
#df_merge

In [20]:
df_merge['fechaBloqueo'] =  pd.to_datetime(df_merge['fechaBloqueo'])
df_merge['timestamp'] =  pd.to_datetime(df_merge['timestamp'])
df_merge.dtypes

Usuarios                         object
item_id                           int64
timestamp           datetime64[ns, UTC]
logtype                          object
title                            object
UsuarioBloqueado                 object
fechaBloqueo        datetime64[ns, UTC]
dtype: object

In [21]:
#libero espacio en la RAM
del arc

In [22]:
filtroFechas =  df_merge['fechaBloqueo'] > df_merge['timestamp']
filtroFechas

0          True
1          True
2          True
3          True
4          True
           ... 
7770908    True
7770909    True
7770910    True
7770911    True
7770912    True
Length: 7770913, dtype: bool

In [23]:
#filtro para que sean actividades previas al bloqueo
dfFiltrado = df_merge[filtroFechas]
dfFiltrado

,Usuarios,item_id,timestamp,logtype,title,UsuarioBloqueado,fechaBloqueo
0,Sabbut,1,2004-12-23 07:10:09+00:00,delete,Verificacionismo,Sabbut,2010-12-15 13:07:57+00:00
1,Sabbut,1,2004-12-23 07:10:09+00:00,delete,Verificacionismo,Sabbut,2010-12-15 13:08:14+00:00
2,Sabbut,2,2004-12-23 07:14:09+00:00,delete,Rapunzel,Sabbut,2010-12-15 13:07:57+00:00
3,Sabbut,2,2004-12-23 07:14:09+00:00,delete,Rapunzel,Sabbut,2010-12-15 13:08:14+00:00
4,Sabbut,3,2004-12-23 07:26:10+00:00,delete,Macromedia Flash,Sabbut,2010-12-15 13:07:57+00:00
...,...,...,...,...,...,...,...
7770908,The Boys Studios,15586495,2021-09-02 18:44:32+00:00,newusers,Usuario:The Boys Studios,The Boys Studios,2021-09-03 00:18:14+00:00
7770909,The Boys Studios,15586508,2021-09-02 18:48:35+00:00,create,Noche De Odio,The Boys Studios,2021-09-03 00:18:14+00:00
7770910,Pervertidosinremedio,15586995,2021-09-02 23:11:30+00:00,newusers,Usuario:Pervertidosinremedio,Pervertidosinremedio,2021-09-02 23:13:25+00:00
7770911,JohnWayne454545,15587001,2021-09-02 23:14:35+00:00,newusers,Usuario:JohnWayne454545,JohnWayne454545,2021-09-03 00:39:46+00:00


In [24]:
#libero RAM
del df_merge

In [25]:
grupo=dfFiltrado.groupby(['Usuarios']).agg({'Usuarios':'count'})
grupo

,Usuarios
Usuarios,
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,1
!! !!ПОМОГИТЕ СЕСТРЕ НА ОПЕРАЦИЮ!! !!,1
"!""·$%&",1
!23455666A!wee,1
!2o%Uv6PHBj6JfgYmfgqVx!SCEBEP4,1
...,...
𝓒𝓸𝓻𝓸𝓷𝓪𝓭𝓸𝓼' 𝓢𝓮𝓻𝓿𝓮𝓻,2
𝕊𝟜𝕒𝕟𝕥𝕚𝟚𝟘𝟘𝟟,1
𝙽𝚘𝙹𝚘𝚍𝚊𝚜𝙿𝚒𝚗𝚜𝚑𝚒𝙿𝚞𝚝𝚡,1


In [26]:
#libero RAM
del dfFiltrado

In [27]:
grupo = grupo.rename(columns={'Usuarios':'experiencia'})
grupo

,experiencia
Usuarios,
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,1
!! !!ПОМОГИТЕ СЕСТРЕ НА ОПЕРАЦИЮ!! !!,1
"!""·$%&",1
!23455666A!wee,1
!2o%Uv6PHBj6JfgYmfgqVx!SCEBEP4,1
...,...
𝓒𝓸𝓻𝓸𝓷𝓪𝓭𝓸𝓼' 𝓢𝓮𝓻𝓿𝓮𝓻,2
𝕊𝟜𝕒𝕟𝕥𝕚𝟚𝟘𝟘𝟟,1
𝙽𝚘𝙹𝚘𝚍𝚊𝚜𝙿𝚒𝚗𝚜𝚑𝚒𝙿𝚞𝚝𝚡,1


In [28]:
grupo['categoria'] = grupo['experiencia'].apply(asignarCat)
grupo

,experiencia,categoria
Usuarios,,
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,1,<10
!! !!ПОМОГИТЕ СЕСТРЕ НА ОПЕРАЦИЮ!! !!,1,<10
"!""·$%&",1,<10
!23455666A!wee,1,<10
!2o%Uv6PHBj6JfgYmfgqVx!SCEBEP4,1,<10
...,...,...
𝓒𝓸𝓻𝓸𝓷𝓪𝓭𝓸𝓼' 𝓢𝓮𝓻𝓿𝓮𝓻,2,<10
𝕊𝟜𝕒𝕟𝕥𝕚𝟚𝟘𝟘𝟟,1,<10
𝙽𝚘𝙹𝚘𝚍𝚊𝚜𝙿𝚒𝚗𝚜𝚑𝚒𝙿𝚞𝚝𝚡,1,<10


In [29]:
grupo.reset_index(inplace=True)

In [30]:
grupo.columns

Index(['Usuarios', 'experiencia', 'categoria'], dtype='object')

In [31]:
cantBlockCat = grupo.groupby('categoria').agg({'Usuarios':'count'})

In [32]:
cantBlockCat

,Usuarios
categoria,
10-40,1563
40-100,426
<10,97678
>100,851


In [33]:
cantTotalCat = totalUsuarios.groupby('categoria').agg({'Usuarios':'count'})

In [34]:
cantTotalCat

,Usuarios
categoria,
10-40,5926
40-100,1538
<10,6262394
>100,2989


In [35]:
probabilidades =  cantBlockCat['Usuarios']/cantTotalCat['Usuarios']

### Resultado

In [36]:
probabilidades

categoria
10-40     0.263753
40-100    0.276983
<10       0.015598
>100      0.284711
Name: Usuarios, dtype: float64